# Invoke Bedrock model using LangChain and a zero-shot prompt

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show how to use a LLM to generate an email response to a customer who provided negative feedback on the quality of customer service that they received from the support engineer. 

We will use Anthropic's Claude model provided by Bedrock in this example. We will use the Bedrock version that is integrated with [LangChain](https://python.langchain.com/docs/get_started/introduction.html). LangChain is a framework for developing applications powered by language models. The key aspects of this framework allow us to augment the Large Language Models by chaining together various components to create advanced use cases.

In this notebook we will use the Bedrock API provided by LangChain. The prompt used in this example is called a zero-shot prompt because we are not providing any additional context other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment*.

#### Context
In the previous example `00_generate_w_bedrock.ipynb`, we explored how to use Boto3 client to communicate with Amazon Bedrock API. In this notebook, we will try to add a bit more complexity to leverage the LangChain framework for the similar use case. We will explore the use of Amazon Bedrock integration within LangChain framework and how it could be used to generate text with the help of `PromptTemplate`.

#### Pattern
We will simply provide the LangChain implementation of Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/bedrock_langchain.jpg)

#### Use Case
To demonstrate the generation capability of models in Amazon Bedrock, let's take the use case of email generation.

#### Persona
You are Bob a Customer Service Manager at AnyCompany and some of your customers are not happy with the customer service and are providing negative feedbacks on the service provided by customer support engineers. Now, you would like to respond to those customers humbly aplogizing for the poor service and regain trust. You need the help of an LLM to generate a bulk of emails for you which are human friendly and personalized to the customer's sentiment from previous email correspondence.

#### Implementation
To fulfill this use case, in this notebook we will show how to generate an email with a thank you note based on the customer's previous email. We will use the Amazon Titan Text Large model using the Amazon Bedrock LangChain integration. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Invoke the Bedrock client using LangChain Integration

Lets begin with creating an instance of Bedrock class from llms. This expects a `model_id` of the model available in Amazon Bedrock. 

Optionally you can pass on a previously created boto3 `client` as well as some `model_kwargs` which can hold parameters such as `temperature`, `topP`, `maxTokenCount` or `stopSequences` (more on parameters can be explored in Amazon Bedrock console).

Check [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) for Available text generation model Ids under Amazon Bedrock.

Note that different models support different `model_kwargs`.

In [3]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

By passing the `client` in to LangChain, we should be able to ensure that the library uses the same boto3 client we checked the configuration of earlier:

In [4]:
print(boto3_bedrock)
print(textgen_llm.client)

LangChain has abstracted away the Amazon Bedrock API and made it easy to build use cases. You can pass in your prompt and it is automatically routed to the appropriate API to generate the response. You simply get the text output as-is and don't have to extract the results out of the response body.

Let's prepare the prompt to generate an email for the Customer Service Manager to send to the customer.

In [5]:
response = textgen_llm("""

Human: Write an email from Bob, Customer Service Manager, 
to the customer "John Doe" that provided negative feedback on the service 
provided by our customer support engineer.

Assistant:""")

print_ww(response)

 Dear Mr. Doe,

I'm writing in regards to your recent experience with our customer support team. I understand you
were dissatisfied with the service you received and I sincerely apologize for that.

At ABC Company, our top priority is providing exceptional customer service. When we fall short of
that goal, I want to know about it so we can make things right.

I have spoken with the support engineer that assisted you and addressed your concerns. I have also
taken steps to provide additional training to our team to help prevent similar issues going forward.

We value you as a customer and want to regain your trust. Please let me know if there is anything
else I can do to make this right. I would be happy to offer you a discount on your next purchase or
provide a refund for the inconvenience you experienced.

Thank you for taking the time to share your feedback. We appreciate the opportunity to improve. I
hope we can demonstrate our commitment to quality service on your future interaction

## Conclusion
You have now experimented with using `LangChain` framework which provides an abstraction layer on Amazon Bedrock API. Using this framework you have seen the usecase of generating an email responding to a customer due to their negative feedback.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the different parameters to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You